In [3]:
import pandas as pd

# -> Betrachtung des Szenario autonom als Grundlage
gemis_daten_benzin = dict(
    thg = 2743,  # g/L
    nox = 1.4,  # g/L
    pm = 0.16,  # g/L
    energieverbrauch = 34.9  # MJ/L 
)

gemis_daten_diesel = dict(
    thg = 3062,  # g/L
    nox = 4.3,  # g/L
    pm = 0.19,  # g/L
    energieverbrauch = 37.4  # MJ/L 
)

gemis_data = pd.concat(
    [pd.DataFrame.from_dict(gemis_daten_benzin, orient='index', columns=['benzin']),
    pd.DataFrame.from_dict(gemis_daten_diesel, orient='index', columns=['diesel'])], axis=1, sort=False)

emissionsdaten = pd.read_hdf('../data/data.h5', key='Bevoelkerungs_und_Emissiondaten')

emissionen = (emissionsdaten.loc[:, 2050].reindex(['spez_Emissionsfaktor_THG', 'spez_Emissionsfaktor_NOx', 'spez_Emissionsfaktor_PM'])
                 .rename({
                     'spez_Emissionsfaktor_THG': 'thg', # g/MJ
                     'spez_Emissionsfaktor_NOx': 'nox', # g/MJ
                     'spez_Emissionsfaktor_PM': 'pm'})) # g/MJ
emissionen.loc['energieverbrauch'] = 1  # MJ
gemis_data.loc[:, 'elektro'] = emissionen

verbrauch = {
    'n1': [9.8/100, 0, 25/100],  # Diesel, Benzin, Elektro l/km
    'n3': [34.5/100, 0, 130/100],  # Diesel, Benzin, Elektro l/km
    'stadtbus': [44.9/100, 0, 240/100],  # Diesel, Benzin, Elektro l/km
    'pro_kg': [0.4/100, 0.3/100,0]   # Diesel, Benzin, Elektro l/km*kg
}
verbrauch = pd.DataFrame.from_dict(verbrauch, orient='index', columns=['diesel', 'benzin', 'elektro'])

verbrauch.to_hdf('../data/data.h5', format="table", key='spezifischer_verbrauch')
gemis_data.to_hdf('../data/data.h5', format="table", key='gemis_data')

In [9]:
11/9.8

1.1224489795918366

In [10]:
25*(11/9.8)

28.061224489795915

In [ ]:
sns.set(context='paper')
sns.set_palette("Set1", n_colors=8, desat=.5)
plt.rcParams.update({'font.size': 14})

df = rf_pv_basis.copy()

av_speed_name = [f'{mode}_av_speed' for mode in unique_modes_fitted]
times_name = [f'{mode}_time' for mode in unique_modes_fitted]
df = pd.concat([df,pd.DataFrame(columns=av_speed_name+times_name)], sort=False)

def av_speed_calc(row):
    if row['only_mode']:
        row.loc[row['main_mode'] + '_av_speed'] = row.loc[row['main_mode']] / row['fahrtzeit'] * 0.06
    return row

df = df.apply(lambda row: av_speed_calc(row), axis=1)

solo_speeds = (df.loc[:,av_speed_name]
               .mean()
               .dropna()
               .to_dict())

solo_speeds['zu_fuss'] = solo_speeds.pop('zu_fuss_av_speed')
solo_speeds['e_bike'] = solo_speeds.pop('e_bike_av_speed')
solo_speeds['fahrrad'] = solo_speeds.pop('fahrrad_av_speed')

def calc_speeds(solo_speeds, df_start):
    df = df_start.copy()
    for key,val in solo_speeds.items():
        df[f'{key}_time'] = df[key] / val
    return df

df = calc_speeds(solo_speeds, df)
tmp = df.reindex(columns= av_speed_name + ['only_mode', 'strecken_typ'])
tmp = tmp.loc[ tmp.loc[:, 'only_mode'] == True ].dropna(axis=1, how='all').drop(columns='only_mode')
grouped = tmp.groupby('strecken_typ')

#display(HTML(grouped.describe().T.to_html()))
#
#with open('../tables/speeds.tex', 'w') as tf:
#     tf.write(grouped.describe().T.to_latex())
#
#grouped.boxplot()
#
#fig = matplotlib.pyplot.gcf()
#fig.set_size_inches(10, 5)
#fig.savefig('../plots/speeds.png', dpi=100)